## SRJ - H generation using qiskit

In [1]:
import numpy as np
from pyscf import ao2mo, gto, mcscf, scf
import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

In [2]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

In [2]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')



# Your original setup (unchanged)
#mol_pyscf = gto.M(
#    atom = 'Fe 0 0 0; S 0 0 1.826',
#    basis = basis_dict,
#    spin = 4)
#rhf = scf.UHF(mol_pyscf)
#rhf.kernel()


distance = 1.826
a = distance
mol = gto.Mole()
mol.build(
    verbose=0,
    atom=[
        ["Fe", (0, 0, 0)],
        ["S", (0, 0, a)],
    ],
    basis=basis_dict,
    spin=4,
    charge=0
)

mf = scf.UHF(mol)
mf.scf()
 
print(
    mf.energy_nuc(),
    mf.energy_elec()[0],
    mf.energy_tot(),
    mf.energy_tot() - mol.energy_nuc(),
)

120.55734925669225 -1775.6589029665963 -1655.101553709904 -1775.6589029665963


In [6]:
active_space = range(mol.nelectron // 2 - 1, mol.nelectron // 2 + 1)
print("Active space:", active_space)

Active space: range(20, 22)


In [7]:
mo_indices = list(range(18, 24))  # This gives [18, 19, 20, 21, 22, 23]


In [9]:
print(mf.mo_energy)


[[-2.59140685e+02 -9.17702594e+01 -3.17701606e+01 -2.74436587e+01
  -2.74436587e+01 -2.73873926e+01 -8.98201592e+00 -6.67311279e+00
  -6.67311279e+00 -6.66666890e+00 -4.31798901e+00 -2.94794200e+00
  -2.94794200e+00 -2.90378322e+00 -1.02532906e+00 -7.78744375e-01
  -7.02992200e-01 -7.02992200e-01 -6.50702012e-01 -4.52811910e-01
  -4.52811910e-01 -4.43685960e-01 -2.60652238e-01  4.36626247e-02
   1.12654303e-01  1.12654303e-01  3.24757216e-01]
 [-2.59137930e+02 -9.17572561e+01 -3.17144635e+01 -2.73849315e+01
  -2.73849315e+01 -2.73405669e+01 -8.96747981e+00 -6.65518227e+00
  -6.64350270e+00 -6.64350270e+00 -4.15663731e+00 -2.75144959e+00
  -2.73526633e+00 -2.73526633e+00 -8.90270823e-01 -5.51488097e-01
  -4.39297978e-01 -4.39297978e-01 -3.86931565e-01 -3.05774381e-03
  -3.05774381e-03  7.54172223e-02  9.58153890e-02  1.32580777e-01
   1.93177267e-01  1.93177267e-01  3.86830073e-01]]


In [8]:
from pyscf.mcscf import CASSCF

ncas = 6              # Number of active orbitals
nelecas = 6           # Number of active electrons (change as needed)

mc = CASSCF(mf, ncas, nelecas)

# Sort the orbitals so your chosen ones are active
mo_coeff = mc.sort_mo(mo_indices)
mc.kernel(mo_coeff=mo_coeff)
print(mf.mo_energy)


(-1655.3743218741802,
 -11.100646478342924,
 FCIvector([[-0.13264553,  0.53294977,  0.10297727, -0.04861154,
             -0.59148706,  0.04384075],
            [ 0.03853699, -0.31666581, -0.06208849,  0.0059257 ,
              0.35641997,  0.11956659],
            [ 0.03878012, -0.06064791, -0.01009534,  0.0799424 ,
              0.03997904,  0.00986951],
            [ 0.00795476,  0.01523757, -0.05706683,  0.00538217,
             -0.03269   , -0.0163674 ],
            [-0.05204711,  0.04307108, -0.01591946, -0.03435709,
              0.16019989,  0.06325044],
            [-0.13250285,  0.10048766,  0.0166123 , -0.03689717,
             -0.05750705, -0.00947539]]),
 array([[ 9.99956495e-01,  3.19982260e-06, -8.67466626e-03,
          1.20838363e-05,  1.38177382e-08,  9.56025617e-09,
         -2.35710200e-05,  3.09610444e-05, -1.72047976e-08,
         -6.63904580e-11,  3.07042343e-03, -1.13817359e-04,
         -1.84591268e-08,  4.72931584e-10, -2.48864591e-03,
          2.45691152e-08

In [13]:
E1 = mf.kernel()
mx = mcscf.CASSCF(mf, ncas=6, nelecas=6)
mo = mx.sort_mo(mo_indices, base=0)
E2 = mx.kernel(mo)[:2]

h1e, ecore = mx.get_h1eff()
h2e = ao2mo.restore(1, mx.get_h2eff(), mx.ncas)

H = build_hamiltonian(ecore, h1e, h2e)
print(H)

NameError: name 'build_hamiltonian' is not defined

Original active space transformer code of qiskit

In [6]:
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(atom="Li 0 0 0; H 0 0 1.5")

full_problem = driver.run()
print(full_problem.num_particles)
print(full_problem.num_spatial_orbitals)

from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

as_transformer = ActiveSpaceTransformer(2, 2)

as_problem = as_transformer.transform(full_problem)

## Working code

In [18]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()


driver = PySCFDriver(
    atom="Fe 0 0 0; S 0 0 1.826",
    basis=basis_dict,
    charge=0,
    spin=4,
    unit=DistanceUnit.ANGSTROM,
)



problem = driver.run()
#problem.num_active_electrons = (3, 3) # (alpha_electrons, beta_electrons)
#problem.num_active_orbitals = 2

as_transformer = ActiveSpaceTransformer(6,6)

as_problem = as_transformer.transform(problem)


HF_energy = as_problem.reference_energy
print(f"Hartree-Fock energy: {HF_energy} Hartree")


fermionic_op = as_problem.hamiltonian.second_q_op()
qubit_jw_op = mapper.map(fermionic_op)
print(qubit_jw_op)


Hartree-Fock energy: -1655.3496838362307 Hartree
SparsePauliOp(['IIIIIIIIIIII', 'IIIIIIIIIIIZ', 'IIIIIIIIIYZY', 'IIIIIIIIIXZX', 'IIIIIIIIYZZY', 'IIIIIIIIXZZX', 'IIIIIIIYZZZY', 'IIIIIIIXZZZX', 'IIIIIIYZZZZY', 'IIIIIIXZZZZX', 'IIIIIIIIIIZI', 'IIIIIIIIIZII', 'IIIIIIIIZIII', 'IIIIIIIZIIII', 'IIIIIIZIIIII', 'IIIIIZIIIIII', 'IIIYZYIIIIII', 'IIIXZXIIIIII', 'IIYZZYIIIIII', 'IIXZZXIIIIII', 'IYZZZYIIIIII', 'IXZZZXIIIIII', 'YZZZZYIIIIII', 'XZZZZXIIIIII', 'IIIIZIIIIIII', 'IIIZIIIIIIII', 'IIZIIIIIIIII', 'IZIIIIIIIIII', 'ZIIIIIIIIIII', 'IIIIIIIIIIZZ', 'IIIIIIIIIZIZ', 'IIIIIIIIZIIZ', 'IIIIIIIZIIIZ', 'IIIIIIZIIIIZ', 'IIIIIZIIIIIZ', 'YZZZZYIIIIIZ', 'XZZZZXIIIIIZ', 'IIIIZIIIIIIZ', 'IIIZIIIIIIIZ', 'IIZIIIIIIIIZ', 'IZIIIIIIIIIZ', 'ZIIIIIIIIIIZ', 'IIIIIIYZZZIY', 'IIIIIIXZZZIX', 'IIIIIIIYYIYY', 'IIIIIIIXXIYY', 'IIIIIIIYYIXX', 'IIIIIIIXXIXX', 'IIIIYYIIIIYY', 'IIIIXXIIIIYY', 'IIIIYYIIIIXX', 'IIIIXXIIIIXX', 'YZZZYIIIIIYY', 'XZZZXIIIIIYY', 'YZZZYIIIIIXX', 'XZZZXIIIIIXX', 'IYYIIIIIIIYY', 'IXXIIIIIIIYY', 'IYYIIII

## Conversion of qiskit to pennylane H

In [19]:
pennylane_op = qml.from_qiskit_op(qubit_jw_op)
print(pennylane_op)

H = pennylane_op


(-9.821531077391308+0j) * I(0) + (0.1512819998732594+0j) * Z(0) + (1.3135292589765623e-07+0j) * (Y(0) @ Z(1) @ Y(2)) + (1.3135292589765623e-07+0j) * (X(0) @ Z(1) @ X(2)) + (2.7544732164985217e-08+0j) * (Y(0) @ Z(1) @ Z(2) @ Y(3)) + (2.7544732164985217e-08+0j) * (X(0) @ Z(1) @ Z(2) @ X(3)) + (-9.707235936295902e-08+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + (-9.707235936295902e-08+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + (-0.021665251184996258+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5)) + (-0.021665251184996258+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5)) + (0.14056203679848672+0j) * Z(1) + (0.1405620367984778+0j) * Z(2) + (0.1183000933430095+0j) * Z(3) + (0.1183000925840902+0j) * Z(4) + (0.04467524440424526+0j) * Z(5) + (0.1512819998732592+0j) * Z(6) + (1.3135292589771044e-07+0j) * (Y(6) @ Z(7) @ Y(8)) + (1.3135292589771044e-07+0j) * (X(6) @ Z(7) @ X(8)) + (2.754473216498521e-08+0j) * (Y(6) @ Z(7) @ Z(8) @ Y(9)) + (2.754473216498521e-08+0j) * (X(6) @ Z(7) @ Z(8) @ X(9)) + (-9.

In [20]:
import pennylane as qml
from pennylane import qchem
electrons = 6
qubits = 12
hf_state = qchem.hf_state(electrons, qubits)
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit():
    qml.BasisState(hf_state, wires=range(qubits))
    return qml.expval(H)

energy = circuit()
print("Energy from PennyLane circuit:", energy)

Energy from PennyLane circuit: -10.957604137036363
